# 基于昇思MindQuantum，实现量子蒙特卡洛算法

#### 本项目主要基于Quantum Computing Quantum Monte Carlo （arXiv:2206.10431v1)的思路实现。

该文章基于Fermion Monte Carlo without fixed nodes: A game of life, death, and annihilation in Slater determinant space, J. Chem. Phys. 131, 054106 (2009), 将经典计算机上的量子蒙特卡洛方法迁移到了量子计算机上。

安装所需要的包（ModelArts Notebook环境自带的包这里已经跳过，如有需要请自行先安装）

In [ ]:
!pip install --upgrade pip -i https://pypi.tuna.tsinghua.edu.cn/simple

In [ ]:
!pip install https://hiq.huaweicloud.com/download/mindquantum/newest/linux/mindquantum-master-cp37-cp37m-linux_x86_64.whl -i https://pypi.tuna.tsinghua.edu.cn/simple

In [ ]:
!pip install openfermionpyscf -i https://pypi.tuna.tsinghua.edu.cn/simple

导入所需要的包

In [ ]:
import numpy as np
import math
import mindspore as ms
from mindspore.common.parameter import Parameter
import mindspore.context as context

from openfermion.chem import MolecularData
from openfermionpyscf import run_pyscf
from mindquantum.core.gates import X
from mindquantum.core.circuit import Circuit, pauli_word_to_circuits, controlled, dagger
from mindquantum.core.operators import Hamiltonian
from mindquantum.simulator import Simulator
from mindquantum.framework import MQAnsatzOnlyLayer
from mindquantum.algorithm.nisq import generate_uccsd

context.set_context(mode=context.PYNATIVE_MODE, device_target="CPU")

## 一、量子化学准备，VQE

定义所要求解的分子的几何构型，这里以直线型的H4分子为例。

In [5]:
dist = 2.0
geometry = [
    ["H", [0.0, 0.0, 1.0 * dist]],
    ["H", [0.0, 0.0, 2.0 * dist]],
    ["H", [0.0, 0.0, 3.0 * dist]],
    ["H", [0.0, 0.0, 4.0 * dist]],
]
basis = "sto3g"
spin = 0
print("Geometry: \n", geometry)

Geometry: 
 [['H', [0.0, 0.0, 2.0]], ['H', [0.0, 0.0, 4.0]], ['H', [0.0, 0.0, 6.0]], ['H', [0.0, 0.0, 8.0]]]


利用openfermion(pyscf)自带的方法，计算出分子基态的Hartree-Fock, CCSD, FCI能量。其中FCI能量即为分子基态能量的精确值。

In [6]:
molecule_of = MolecularData(
    geometry,
    basis,
    multiplicity=2 * spin + 1
)
molecule_of = run_pyscf(
    molecule_of,
    run_scf=1,
    run_ccsd=1,
    run_fci=1
)

print("Hartree-Fock energy: %20.16f Ha" % (molecule_of.hf_energy))
print("CCSD energy: %20.16f Ha" % (molecule_of.ccsd_energy))
print("FCI energy: %20.16f Ha" % (molecule_of.fci_energy))

Hartree-Fock energy:  -1.5756164767018666 Ha
CCSD energy:  -1.9160861180298063 Ha
FCI energy:  -1.8977806459898732 Ha


保存计算结果，供后续调用

In [7]:
molecule_of.save()
molecule_file = molecule_of.filename
print(molecule_file)

/home/ma-user/anaconda3/envs/MindSpore/lib/python3.7/site-packages/openfermion/testing/data/H4_sto3g_singlet


构建VQE的初态，也即hartree-fock态，即在编号前molecule_of.n_electrons（也即总量子比特数的一半）个qubit上作用一个X门。

In [8]:
hartreefock_wfn_circuit = Circuit([X.on(i) for i in range(molecule_of.n_electrons)])
print(hartreefock_wfn_circuit)

q0: ──X──

q1: ──X──

q2: ──X──

q3: ──X──


通过MindQuantum自带的方法构造基于uccsd的VQE线路ansatz

In [9]:
ansatz_circuit, \
init_amplitudes, \
ansatz_parameter_names, \
hamiltonian_QubitOp, \
n_qubits, n_electrons = generate_uccsd(molecule_file)

ccsd:-1.9160861180298063.
fci:-1.8977806459898732.


由于制备hartree-fock态的线路不含参数，可以将hartreefock_wfn_circuit与ansatz_circuit合并，方便后续使用MQAnsatzOnlyLayer这一非常便捷的功能。

In [10]:
total_circuit = hartreefock_wfn_circuit + ansatz_circuit
total_circuit.summary()
print("Number of parameters: %d" % (len(ansatz_parameter_names)))

==========================Circuit Summary==========================
|Total number of gates  : 3332.                                   |
|Parameter gates        : 160.                                    |
|with 14 parameters are :                                         |
|p0, p4, p1, p5, p2, p6, p3, p7, p8, p9..                        .|
|Number qubit of circuit: 8                                       |
Number of parameters: 14


定义模拟器和梯度更新算子。

In [11]:
sim = Simulator('mqvector', total_circuit.n_qubits)
molecule_pqc = sim.get_expectation_with_grad(Hamiltonian(hamiltonian_QubitOp), total_circuit)
molecule_pqcnet = MQAnsatzOnlyLayer(molecule_pqc, 'Zeros')
initial_energy = molecule_pqcnet()
print("Initial energy: %20.16f" % (initial_energy.asnumpy()))

Initial energy:  -1.5756164789199829


使用MindSpore的Adagrad算法进行梯度优化，标准的VQE流程。

In [12]:
optimizer = ms.nn.Adagrad(molecule_pqcnet.trainable_params(), learning_rate=4e-2)
train_pqcnet = ms.nn.TrainOneStepCell(molecule_pqcnet, optimizer)

eps = 1.e-8
energy_diff = eps * 1000
energy_last = initial_energy.asnumpy() + energy_diff
iter_idx = 0
while abs(energy_diff) > eps:
    energy_i = train_pqcnet().asnumpy()
    if iter_idx % 5 == 0:
        print("Step %3d energy %20.16f" % (iter_idx, float(energy_i)))
    energy_diff = energy_last - energy_i
    energy_last = energy_i
    iter_idx += 1

print("Optimization completed at step %3d" % (iter_idx - 1))
print("Optimized energy: %20.16f" % (energy_i))
print("Optimized amplitudes: \n", molecule_pqcnet.weight.asnumpy())

Step   0 energy  -1.5756164789199829
Step   5 energy  -1.7936779260635376
Step  10 energy  -1.8394664525985718
Step  15 energy  -1.8595349788665771
Step  20 energy  -1.8704031705856323
Step  25 energy  -1.8769905567169189
Step  30 energy  -1.8812921047210693
Step  35 energy  -1.8842602968215942
Step  40 energy  -1.8863967657089233
Step  45 energy  -1.8879827260971069
Step  50 energy  -1.8891848325729370
Step  55 energy  -1.8901070356369019
Step  60 energy  -1.8908178806304932
Step  65 energy  -1.8913655281066895
Step  70 energy  -1.8917859792709351
Step  75 energy  -1.8921068906784058
Step  80 energy  -1.8923504352569580
Step  85 energy  -1.8925340175628662
Step  90 energy  -1.8926718235015869
Step  95 energy  -1.8927748203277588
Step 100 energy  -1.8928515911102295
Step 105 energy  -1.8929085731506348
Step 110 energy  -1.8929510116577148
Step 115 energy  -1.8929824829101562
Step 120 energy  -1.8930058479309082
Step 125 energy  -1.8930232524871826
Step 130 energy  -1.8930362462997437
S

可以看到上述的最优化能量-1.89307与精确值-1.89778仍有大于化学精度的误差。在VQE结果的基础上，我们进行量子蒙特卡洛算法Quantum Computing Quantum Monte Carlo (QQMC)的实现。

## 二、量子蒙特卡洛，QQMC

将上述计算得到的最优化参数导入VQE线路，作为U，需要注意这里的U不包含Hartree-Fock态的制备线路，只有ansatz

In [13]:
U = ansatz_circuit.apply_value(dict(zip(ansatz_parameter_names, list(molecule_pqcnet.weight.asnumpy()))))
Udag = dagger(U)

在论文中，作者使用一个量子线路来估计相似变换后的哈密顿量在qubit basis下的矩阵元。但在模拟器中，我们可以直接计算出这些矩阵元。将之前计算出的hamiltonian_QubitOp利用MindQuantum自带的split方法逐项拆分为一系列单个的Pauli串，将每一项先用pauli_word_to_circuits方法转化成Circuit类型，再用Circuit类型自带的matrix方法即可导入Pauli串对应的矩阵。将这些矩阵依照split时的权重重新加和，即可得到完整的哈密顿量。

H4分子的哈密顿量还较小，我们可以做一个简单的验证，将其对角化与前文中的FCI能量做对比，发现是一致的。

In [14]:
p = list(hamiltonian_QubitOp.split())
emptycirc = Circuit()
for i in range(n_qubits):
    emptycirc += X.on(i)
    emptycirc += X.on(i)
a = pauli_word_to_circuits(p[0][1])
circ = emptycirc + a
hamatrix = p[0][0].const * circ.matrix()
n_pau = len(p)
for i in range(1, n_pau):
    a = pauli_word_to_circuits(p[i][1])
    circ = emptycirc + a
    hamatrix += p[i][0].const * circ.matrix()
hamatrix = np.dot(np.dot(Udag.matrix(), hamatrix),U.matrix())
hamatrix = hamatrix.real

e, v = np.linalg.eig(hamatrix)
e.sort()
print(e[0])

-1.8977806459900022


对于较大的哈密顿量，这一计算可能耗时较长，可以启用以下的两个cell
中的四行代码存储矩阵文件，在后续使用时直接调用文件。

In [15]:
# with open('H.npy', 'wb') as f:
    # np.save(f, hamatrix)

In [16]:
# with open('h4.npy', 'rb') as f:
    # hamatrix = np.load(f)

可以将制备Hartree-Fock态的线路也看做一个酉变换。我们因此可以知道，如果定义一个二进制串00...0，将最低位的，数量为qubit数目一半的0翻转为1，得到的数会对应于Hartree-Fock态在哈密顿量的qubit basis表象下的量子态。在H4分子中，即对应第00001111个，也即第15个量子态。

我们也可以做一个简单地验证，输出hamatrix[15,15]这个矩阵元，发现它与前文计算得到的Hartree-Fock能量是一致的。

In [26]:
hf = 2 ** molecule_of.n_electrons - 1

print(hamatrix[hf,hf])

-1.8930739379526567


接下来是QMC的主体，参考论文中的Algorithm 1 QC-FCIQMC，但需要注意的是，论文有多处错误：

1）“Label the new walker $|\phi_j \rangle ...$”这一步，需要在两符号相乘的基础上再乘上一个负号，原因是式(6)的等式右端本身就带有负号

2）没有给出S的更新方式。更新方式实际我参考了https://zhuanlan.zhihu.com/p/369346554 中的在每$A$个循环后，$S(\tau)=S(\tau-A \delta \tau)-\frac{\zeta}{A \delta \tau} \ln \frac{N_{\mathrm{w}}(\tau)}{N_{\mathrm{w}}(\tau-A \delta \tau)}$，其中损耗因子$\zeta$我设置为了1

3）没有给出更新S的条件。更新条件我参考了论文中的控制popolation至10000，设置为了每10步判断总population是否超过10000，在超过是更新S

4）同种反号walker相互湮灭这一步骤，将for $i$ in $\mathcal{D}$ do错误地写为了in $S$

5）这一步骤循环的位置也写错了，应该写在与上一个for $i$ in $\mathcal{D}$同级的地方，仅在n的循环的内层。

In [31]:
T = 700
dt = 0.01
zeta = 1.0
A = 10
maxpop = 10000

D = set([hf])
Nmax = 2 ** n_qubits
pospop = [0 for i in range(Nmax)]
pospop[hf] = 1
negpop = [0 for i in range(Nmax)]
Slast = 0
S = 0
Nlast = 1
N = 1
for n in range(T):
    toadd = []
    for i in D:
        if max(pospop[i], negpop[i]) == 0:
            continue
        isign = 1
        if negpop[i] > 0:
            isign = -1
        for w in range(Nmax):
            if w == i:
                hii = hamatrix[i,i]
                p_i = dt * (hii - S)
                ran = np.random.uniform(0, 1, max(pospop[i], negpop[i]))
                n_new = np.sum(ran < np.abs(p_i))
                if n_new > 0:
                    if p_i < 0:
                        if isign > 0:
                            pospop[i] += n_new
                        else:
                            negpop[i] += n_new
                        N += n_new
                    else:
                        if isign > 0:
                            pospop[i] -= n_new
                        else:
                            negpop[i] -= n_new
                        N -= n_new
                continue
            Hjisign = 1
            Hji = hamatrix[w,i]
            if Hji < 0:
                Hjisign = -1
            if np.abs(Hji) > 1e-8:
                ran = np.random.uniform(0, 1, max(pospop[i], negpop[i]))
                prob = dt * np.abs(Hji)
                n_new = np.sum(ran < prob)
                if n_new > 0:
                    toadd.append(w)
                    if isign * Hjisign > 0:
                        negpop[w] += n_new
                    else:
                        pospop[w] += n_new
                    N += n_new
    for i in D:
        if pospop[i] > negpop[i]:
            pospop[i] -= negpop[i]
            N -= 2 * negpop[i]
            negpop[i] = 0
        else:
            negpop[i] -= pospop[i]
            N -= 2 * pospop[i]
            pospop[i] = 0
            
    if n % A == A - 1:
        if N > maxpop:
            S = Slast - zeta / A / dt * np.log(N / Nlast)
            Slast = S
        Nlast = N
    
    for i in toadd:
        D.add(i)

    if n % 50 == 49:
        temp = sum(pospop) + sum(negpop)
        mixed_energy = energy_i[0]
        for i in range(Nmax):
            if i != 15:
                if pospop[i] > 0:
                    mixed_energy += hamatrix[i,hf] * pospop[i] / pospop[hf]
                else:
                    mixed_energy -= hamatrix[i,hf] * negpop[i] / pospop[hf]
        print('step %d: mixed energy = %.10f, total pupulation = %d' % (n + 1, mixed_energy, temp))
        
mixed_energy = energy_i[0]

for i in range(Nmax):
    if i != 15:
        if pospop[i] > 0:
            mixed_energy += hamatrix[i,hf] * pospop[i] / pospop[hf]
        else:
            mixed_energy -= hamatrix[i,hf] * negpop[i] / pospop[hf]
    
print('VQE: %.10f' % energy_i[0])
print('QQMC: %.10f' % mixed_energy)
print('FCI: %.10f' % molecule_of.fci_energy)

step 50: mixed energy = -1.8930737972, total pupulation = 1
step 100: mixed energy = -1.8930737972, total pupulation = 1
step 150: mixed energy = -1.8930737972, total pupulation = 3
step 200: mixed energy = -1.8930737972, total pupulation = 14
step 250: mixed energy = -1.8970708654, total pupulation = 72
step 300: mixed energy = -1.8967936314, total pupulation = 191
step 350: mixed energy = -1.8966750774, total pupulation = 521
step 400: mixed energy = -1.8966766404, total pupulation = 1286
step 450: mixed energy = -1.8965401927, total pupulation = 3283
step 500: mixed energy = -1.8966937984, total pupulation = 8485
step 550: mixed energy = -1.8969660751, total pupulation = 10304
step 600: mixed energy = -1.8972964796, total pupulation = 10272
step 650: mixed energy = -1.8976165738, total pupulation = 10254
step 700: mixed energy = -1.8979787387, total pupulation = 10279
VQE: -1.8930737972
QQMC: -1.8979787387
FCI: -1.8977806460


可以发现，QQMC的结果比VQE的结果更接近FCI的精确值，验证了算法的有效性。

此外，若将U改为空线路，则实现了经典计算机上的量子蒙特卡洛FCIQMC

## 三、可调参数

1. 原子/分子构型
2. 演化步数T
3. 演化步长dt
4. 损耗因子zeta
5. S的更新频率A
6. 最大总walker规模maxpop

改变任何参数都只需要在变量的定义处作修改，不需要改变其他代码，会自动相应调整。